# Test my crowding code as if in the driver

In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(0, '/Users/ageller/WORK/LSST/onGitHub/EBLSST/code')
from LSSTEBClusterWorker import LSSTEBClusterWorker
from OpSim import OpSim
from getClusterBinaries import getClusterBinaries

%load_ext autoreload
%autoreload 2

/Users/ageller/anaconda3/lib/python3.7/site-packages/pysynphot/refs.py:118: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for /Users/ageller/python_scripts/PySynphotData/mtab/*_tmg.fits
  'functionality will be SEVERELY crippled. ' + str(e))
/Users/ageller/anaconda3/lib/python3.7/site-packages/pysynphot/refs.py:125: UserWarning: No thermal tables found, no thermal calculations can be performed. No files found for /Users/ageller/python_scripts/PySynphotData/mtab/*_tmt.fits
  'no thermal calculations can be performed. ' + str(e))
/Users/ageller/anaconda3/lib/python3.7/site-packages/numpy/lib/npyio.py:2286: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


In [2]:
GCDF = pd.read_csv("/Users/ageller/WORK/LSST/onGitHub/EBLSST/input/clusters/GCdataForEBLSST.csv").fillna(0.)
field = GCDF.loc[GCDF['ID'] == 'NGC 6254'] #M10 GC
field

,ID,RA[hr],Dec[deg],dist[pc],rhm[pc],mass[Msun],age[Myr],[Fe/H],sigma_v0_z[km/s],OpSimID,OpSimRA[deg],OpSimDec[deg]
59,NGC 6254,16 57 09.05,-04 06 01.1,4400.0,2.495821,177431.202408,10345.238095,-1.56,6.6,2413.0,253.606593,-5.090318


In [3]:
OpS = OpSim()
OpS.dbFile = '/Users/ageller/WORK/LSST/onGitHub/EBLSST/input/db/baseline2018a.db'
OpS.getCursors()

have summary cursor.
have field cursor.


In [4]:
OpS.fieldID = field['OpSimID'].values
OpS.RA = field['OpSimRA[deg]'].values
OpS.Dec = field['OpSimDec[deg]'].values
OpS.obsDates = np.full_like(OpS.fieldID, dict(), dtype=dict)
OpS.NobsDates = np.full_like(OpS.fieldID, dict(), dtype=dict)
OpS.m_5 = np.full_like(OpS.fieldID, dict(), dtype=dict)
OpS.totalNobs = np.full_like(OpS.fieldID, 0)

In [5]:
OpSimi = 0
worker = LSSTEBClusterWorker()
worker.filterFilesRoot = '/Users/ageller/WORK/LSST/onGitHub/EBLSST/input/filters/'
worker.clusterName = field['ID'].values
worker.clusterMass = field['mass[Msun]'].values
worker.clusterDistance = field['dist[pc]'].values/1000.
worker.clusterMetallicity = field['[Fe/H]'].values
worker.clusterAge = field['age[Myr]'].values
worker.clusterRhm = field['rhm[pc]'].values
worker.clusterVdisp = field['sigma_v0_z[km/s]'].values
    
worker.OpSim = OpS

worker.seed = 1111

worker.initialize(OpSimi)

total number of OpSim observation dates (all filters) = 951.0


True

In [6]:
worker.galDir = 'TRILEGALmodels'
worker.getGalaxy(OpSimi)

/Users/ageller/WORK/LSST/onGitHub/EBLSST/code/vespa_update/trilegal.py:99: FutureWarning: read_table is deprecated, use read_csv instead.
  df = pd.read_table(outfile, sep='\s+', skipfooter=1, engine='python')


downloaded TRILEGAL model for ID=2413.0, RA=253.606593, DEC=-5.090318, Nstars=409325.0, Nstars/resEl=0.0006445656966490301


In [8]:
worker.n_bin = 100
clusterDat = worker.sampleCluster(OpSimi)

sampling cluster NGC 6254
initial binary input: 1111 10345.238095238095 -1.56 100 6.6 4.0
Number of binaries 70
!!!!! CHECK THIS: new COSMIC does not give values in the log!


In [15]:
i=1
line= clusterDat[i]
#change the inclination
line[12] = np.pi/2.
#change the period
line[2] = np.log10(3.47)
print(line)
print(worker.EB.crowding.rPlummer)

#!!!!!!NEED TO DEFINE xBinary and yBinary in worker

EB = worker.getEB(line, OpSimi=OpSimi)


print(worker.EB.crowding.backgroundFlux)
print(worker.EB.crowding.backgroundMag)

[0.8375679309455127 0.6207229239740351 0.5403294747908738
 0.010209538408000141 1.0230141030024076 1.0292046948838147
 1.0000095550520773 1.0000183863382084 None None None 4.4
 1.5707963267948966 2.5920156064693947 2.6303476602435496 None -1.56 inf
 inf 11.0 11.0]
1.9128495433910215
getting crowding ... 


NameError: name 'rPlummer' is not defined

In [ ]:
#now plot the crowding
f,ax = plt.subplots(1,6, figsize=(15, 5))
for i,f in enumerate(filters):
    ax[i].pcolormesh(worker.EB.crowding.xgrid, worker.EB.crowding.ygrid, 
                     worker.EB.crowding.fluxgrid[f])#, norm=colors.LogNorm())
    ax[i].set_aspect(1)
    ax[i].set_title(f)
    ax[i].scatter(worker.EB.crowding.clusterSingles['xAng'], 
                  worker.EB.crowding.clusterSingles['yAng'], s=3, color='white')

In [ ]:
#check light_3

#also check the light curve
#worker.run_ellc(0)